In [ ]:
%load_ext autoreload
%autoreload

In [ ]:
import os as _os
_os.chdir(_os.environ['PROJECT_ROOT'])
print(_os.path.realpath(_os.path.curdir))

In [ ]:
import graph_tool as gt
import graph_tool.draw
import numpy as np
import pandas as pd
import scipy.sparse
import scipy as sp
from collections import defaultdict

In [ ]:
# Functions for constructing graphs
def path_to_edgelist(path):
    u = path[0]
    edges = []
    for v in path[1:]:
        edges.append((u, v))
        u = v
    return edges

def new_graph_from_merged_paths(paths, lengths, depths):
    g = gt.Graph()
    for p in paths:
        g.add_edge_list(path_to_edgelist(p))
    g.vp['depth'] = g.new_vp('vector<float>')
    g.vp.depth.set_2d_array(depths)
    g.vp['length'] = g.new_vp('int', lengths)  
    g.gp['nsample'] = g.new_gp('int', len(depths))
    g.vp['sequence'] = g.new_vp('object', vals=[[k] for k in range(g.num_vertices())])
    return g

def get_depth_matrix(g, vs=None):
    if not vs:
        return g.vp.depth.get_2d_array(range(g.gp.nsample))
    else:
        return np.stack([g.vp.depth[i] for i in vs], axis=1)

In [ ]:
def draw_graph(g, **kwargs):
    return gt.draw.graph_draw(g, output_size=(300, 300), ink_scale=0.8, **kwargs)

In [ ]:
paths = [
    [0, 1, 2, 0],
    [3, 4, 5, 3],
    [0, 6, 3],
    [5, 7, 8, 9, 10, 11],
    [9, 12],
    [12, 9],
]

nsamples = 2
nnodes = 13

g0 = new_graph_from_merged_paths(
    paths,
    depths=np.array(np.random.randint(1, 3, size=(nsamples, nnodes))),
    lengths=np.array([1] * 13),
)

In [ ]:
get_depth_matrix(g0)

In [ ]:
_seq_str = g0.new_vp('string', list(g0.vp.sequence))

g0_pos = draw_graph(g0, vertex_text=_seq_str)

In [ ]:
draw_graph(g0, pos=g0_pos, vertex_text=g0.vp.length)

In [ ]:
def edge_has_no_siblings(g):
    "Check whether upstream or downstream sibling edges exist for every edge."
    vs = g.get_vertices()
    v_in_degree = g.new_vertex_property('int', vals=g.get_in_degrees(vs))
    v_out_degree = g.new_vertex_property('int', vals=g.get_out_degrees(vs))
    e_num_in_siblings = gt.edge_endpoint_property(g, v_in_degree, 'target')
    e_num_out_siblings = gt.edge_endpoint_property(g, v_out_degree, 'source')
    e_has_no_sibling_edges = g.new_edge_property('bool', (e_num_in_siblings.a <= 1) & (e_num_out_siblings.a <= 1))
    return e_has_no_sibling_edges

def vertex_does_not_have_both_multiple_in_and_multiple_out(g):
    vs = g.get_vertices()
    return g.new_vertex_property('bool', vals=(
        (g.get_in_degrees(vs) <= 1)
        | (g.get_out_degrees(vs) <= 1)
    ))

# def vertex_does_not_have_multiple_in_or_multiple_out(g):
#     vs = g.get_vertices()
#     return g.new_vertex_property('bool', vals=(
#         (g.get_in_degrees(vs) <= 1)
#         & (g.get_out_degrees(vs) <= 1)
#     ))

def label_maximal_unitigs(g):
    "Assign unitig indices to vertices in maximal unitigs."
    no_sibling_edges = edge_has_no_siblings(g)
    # Since any vertex that has both multiple in _and_ multiple out
    # edges cannot be part of a larger maximal unitig,
    # we could filter out these vertices, at the same time as we
    # are filtering out the edges with siblings.
    # Potentially this would make the component labeling step
    # much faster.
    both_sides_branch = vertex_does_not_have_both_multiple_in_and_multiple_out(g)
    # TODO: Double check, if this has any implications for the
    # "unitig-ness" of its neighbors. I _think_
    # if we mark edges with siblings before filtering out
    # these nodes we should be good.
    g_filt = gt.GraphView(
        g,
        efilt=no_sibling_edges,
        vfilt=both_sides_branch,
        directed=True
    )
    g_filt_undirected = gt.GraphView(g_filt, directed=False)
    # Since we've filtered out the both_sides_branch vertices,
    # the labels PropertyMap would include a bunch of the default value (0)
    # for these. Instead, we set everything not labeled to -1, now a magic
    # value for nodes definitely not in maximal unitigs.
    labels = g.new_vertex_property('int', val=-1)
    labels, counts = gt.topology.label_components(g_filt_undirected, vprop=labels)
    return labels, counts, g_filt

In [ ]:
edge_has_no_siblings(g0).a

In [ ]:
labels, sizes, _g = label_maximal_unitigs(g0)
print(labels.a)

draw_graph(_g, pos=g0_pos, vertex_text=g0.vertex_index)

In [ ]:
draw_graph(g0, vertex_text=label_maximal_unitigs(g0)[0], pos=g0_pos)

In [ ]:
from functools import reduce
import operator

def maximal_unitigs(g):
    "Generate maximal unitigs as lists of vertices"
    # (1a) Filter edges
    # (1b) Label unitigs
    labels, counts, g_filt = label_maximal_unitigs(g)
    # (2) Find every node in the filtered graph without in-edges (these are the origins)
    is_origin = g_filt.new_vp('bool', g_filt.get_in_degrees(g_filt.get_vertices()) == 0)
    # (3) Find every node in the filtered graph without out-edges (these are the termina)
    is_terminus = g_filt.new_vp('bool', g_filt.get_out_degrees(g_filt.get_vertices()) == 0)
    # (4) Iter through the labels.
    for i, c in enumerate(counts):
        # (a) For each, select the subgraph for that unitig.
        vfilt = (labels.a == i)
        assert vfilt.sum() == c
        subgraph = gt.GraphView(g_filt, vfilt=vfilt)
        # (b) Identify the origin a terminus node in the path
        origin = gt.GraphView(subgraph, vfilt=is_origin).get_vertices()
        terminus = gt.GraphView(subgraph, vfilt=is_terminus).get_vertices()
        assert len(origin) == 1
        assert len(terminus) == 1
        origin, terminus = origin[0], terminus[0]
        if origin == terminus:
            continue
        # (c) Trace the route from the origin to the terminus (`graph_tool.topology.all_paths`)
        unitig = list(gt.topology.all_paths(subgraph, origin, terminus))
        assert len(unitig) == 1
        unitig = unitig[0]
        # (d) Ask if, in the unfiltered graph, the terminus connects to the origin. If so, this is a cycle.
        is_cycle = bool(g.edge(terminus, origin))
        # (e) Yield the route and whether it's a cycle.
        yield list(unitig), is_cycle

    
    
#     # The output *labels* is an array matching vertices to their unitig ids
#     # while *counts* matches these ids to their sizes.
#     # Therefore we enumerate the latter to iterate
#     # through the list of maximal unitigs.
#     # NOTE: While label_maximal_unitigs does return the
#     # filtered and undirected graph with sibling edges and multi-in-and-multi-out
#     # vertices removed, the _original_ graph should be used for the unitig
#     # construction below.
#     for i, c in enumerate(counts):
#         vfilt = (labels.a == i)
#         assert vfilt.sum() == c
#         subgraph = gt.GraphView(g, vfilt=vfilt)
#         # TODO: How confident am I that the vs sequence is the correct
#         # output sequence? Are the first/last nodes
#         # always the overlapping nodes?
#         # What happens when it's compressing a cycle?
#         # Turns out the order of vs is simply the
#         # order of node ids (because they're read
#         # off the label array).
#         # TODO: To get the correct order I need to
#         # consider these nodes on the original
#         # graph and find the node that has
#         # multiple downstream neighbors.
#         # This node is the cycle "exit", and
#         # should be at the start (and the end)
#         # of the sequence list.
#         # Also, maybe I can take the most upstream
#         # and most downstream node in the
#         # unitig (the ones with multiple in and multiple
#         # out neighbors), and ask for a path from one
#         # of these upstream neighbors to one of the downstream
#         # neighbors.
#         # The resulting path, with the neighbors trimmed off
#         # is the correct vertex order.
#         unitig = list(subgraph.iter_vertices())
#         is_cycle = not gt.topology.is_DAG(subgraph)
#         yield unitig, is_cycle

In [ ]:
_seq_str = g0.new_vp('string', list(_g.vp.sequence))

draw_graph(g0, pos=g0_pos, vertex_text=g0.vertex_index)

In [ ]:
unitigs, cycles = list(zip(*maximal_unitigs(g0)))
unitigs, cycles

In [ ]:
def list_unitig_neighbors(g, vs):
    "The in and out neighbors of a unitig path."
    all_ins = reduce(operator.add, map(lambda v: list(g.iter_in_neighbors(v)), vs))
    all_outs = reduce(operator.add, map(lambda v: list(g.iter_out_neighbors(v)), vs))
    return list(set(all_ins) - set(vs)), list(set(all_outs) - set(vs))

In [ ]:
list_unitig_neighbors(g0, [3, 4, 5])

In [ ]:
def mutate_add_compressed_unitig_vertex(g, vs, is_cycle, drop_vs=False):
    v = int(g.add_vertex())
    nsample = g.gp.nsample
    in_neighbors, out_neighbors = list_unitig_neighbors(g, vs)
    g.add_edge_list((neighbor, v) for neighbor in in_neighbors)
    g.add_edge_list((v, neighbor) for neighbor in out_neighbors)
    g.vp.length.a[v] = g.vp.length.a[vs].sum()
    g.vp.sequence[v] = reduce(operator.add, (g.vp.sequence[u] for u in vs), [])
    g.vp.depth[v] = (
        (
            get_depth_matrix(g, vs)
            * g.vp.length.a[vs]
        ).sum(1) / g.vp.length.a[v]
    )
    if is_cycle:
        g.add_edge(v, v)
    for old_v in vs:
        g.clear_vertex(old_v)
    return g

In [ ]:
def mutate_compress_all_unitigs(g):
    unitig_list = maximal_unitigs(g)
    all_vs = []
    for i, (vs, is_cycle) in enumerate(unitig_list):
        # TODO: If len(vs) == 1, this is effectively a no-op and can be dropped.
        mutate_add_compressed_unitig_vertex(g, vs, is_cycle)
        all_vs.extend(vs)
    
    g.remove_vertex(set(all_vs), fast=True)
    # I think, but am not sure, that the number of nodes removed will always equal the number of edges removed.
    return g

In [ ]:
g1 = mutate_compress_all_unitigs(g0.copy())

In [ ]:
draw_graph(g0, pos=g0_pos, vertex_text=g0.new_vp('string', list(g0.vp.sequence)))

In [ ]:
g1_pos = draw_graph(g1, vertex_text=g1.vertex_index)

In [ ]:
draw_graph(g1, pos=g1_pos, vertex_text=g1.new_vp('string', list(g1.vp.sequence)))

In [ ]:
get_depth_matrix(g1)

In [ ]:
draw_graph(g1, pos=g1_pos, vertex_text=g1.vp.length)


In [ ]:
from itertools import product
from collections import namedtuple

# FIXME: Don't work with 'l', and 'd' as elements of the Split namedtuple.
Split = namedtuple('Split', ['u', 'v', 'w'])

def all_local_paths_as_splits(g, v):
    "Generate all splits, the product of all in-edges crossed with all out-edges."
    assert v < g.num_vertices(ignore_filter=True)
    us = list(g.iter_in_neighbors(v))
    ws = list(g.iter_out_neighbors(v))
    num_splits = (len(us) * len(ws))
    length = g.vp.length[v]
    depth = get_depth_matrix(g, vs=[v])
    for u, w in product(us, ws):
        # NOTE: This (dummy) splitting function evenly distributes across all paths.
        # FIXME: Casting depth as a tuple is an ugly hack, necessary
        # because I use these Splits as keys into a dictionary.
        yield Split(u, v, w), length, depth / num_splits

def build_tables_from_splits(split_list, start_idx):
    """Generate edges to and from new, split vertices.
    
    Note that if splits from adjacent parents are not
    reciprocated, no new edge is produced.
    
    """
    split_idx = {}
    upstream = defaultdict(list)
    downstream = defaultdict(list)
    length = []
    depth = []
    for idx, (split, l, d) in enumerate(split_list, start=start_idx):
        u, v, w = split
        split_idx[split] = idx
        upstream[(v, w)].append(split)
        downstream[(u, v)].append(split)
        depth.append(d)
        length.append(l)
    return split_idx, upstream, downstream, np.array(length), np.array(depth)
        
        
def new_edges_from_splits(split_list, split_idx, upstream, downstream, start_idx):
    for v, (split, _, _) in enumerate(split_list, start=start_idx):
        u_old, v_old, w_old = split
        v = split_idx[split]
        
        # Upstream edges
        yield (u_old, v, split_idx[split])
        for upstream_split in upstream[(u_old, v_old)]:
            u = split_idx[upstream_split]
            yield (u, v)
            
        # Downstream edges
        yield (v, w_old)
        for downstream_split in downstream[(v_old, w_old)]:
            w = split_idx[downstream_split]
            yield (v, w)

In [ ]:
def mutate_apply_splits(g, split_list):
    """Add edges and drop any parent vertices that were split.
    
    """
    start_idx = g.num_vertices(ignore_filter=True)
    split_idx, upstream, downstream, lengths, depths = (
        build_tables_from_splits(split_list, start_idx=start_idx)
    )
    edges_to_add = list(set(new_edges_from_splits(
        split_list, split_idx, upstream, downstream, start_idx
    )))
    g.add_edge_list(set(edges_to_add))
    g.vp.length.a[np.arange(len(lengths)) + start_idx] = lengths
    new_depth = get_depth_matrix(g)
    new_depth[:, np.arange(len(depths)) + start_idx] = depths.T
    g.vp.depth.set_2d_array(new_depth)
    for split, _, _ in split_list:
        g.vp.sequence[split_idx[split]] = g.vp.sequence[split.v]
    vertices_to_drop = set(split.v for (split, _, _) in split_list)
    g.remove_vertex(vertices_to_drop, fast=True)
    return g

In [ ]:
paths = [
    [3, 4, 5, 0, 1, 2, 6, 7, 8],
    [1, 1],
]

nvertices = 9

_g = new_graph_from_merged_paths(
    paths,
    depths=np.array([[1] * nvertices]),
    lengths=np.array([1] * nvertices),
)
_g_pos = draw_graph(_g, vertex_text=_g.vp.depth)

split_list = list(all_local_paths_as_splits(_g, 1))
print(split_list)
mutate_apply_splits(_g, split_list)
draw_graph(_g, vertex_text=_g.vp.depth)
print(list(_g.vp.sequence))

In [ ]:
draw_graph(g1, pos=g1_pos, vertex_text=g1.vertex_index)
draw_graph(g1, pos=g1_pos, vertex_text=g1.vp.length)
print(get_depth_matrix(g1))

# Split vertex 1, but drop one of the potential splits
# (the one reflecting a linear path with no repeats.)
split_list = list(all_local_paths_as_splits(g1, 0))  #  if split != Split(1, 0, 3)]
print(split_list)
g2 = mutate_apply_splits(g1.copy(), split_list=split_list)
print(get_depth_matrix(g2))
draw_graph(g2, vertex_text=g2.vp.length)

In [ ]:
paths = [
    [0, 1, 2, 0],
    [2, 2]
]
nvertices = 3
_g = new_graph_from_merged_paths(
    paths,
    depths=np.array([[1] * nvertices]),
    lengths=np.array([1] * nvertices),
)

print(get_depth_matrix(_g))
print((get_depth_matrix(_g) * _g.vp.length.a).sum())

draw_graph(_g, vertex_text=_g.vertex_index)

split_list = list(all_local_paths_as_splits(_g, 2))
_g = mutate_apply_splits(_g, split_list=split_list)

print(get_depth_matrix(_g))
print((get_depth_matrix(_g) * _g.vp.length.a).sum())

split_list = list(all_local_paths_as_splits(_g, 2))
_g = mutate_apply_splits(_g, split_list=split_list)

draw_graph(_g, vertex_text=_g.vertex_index)

split_list = list(all_local_paths_as_splits(_g, 2))
_g = mutate_apply_splits(_g, split_list=split_list)
# # draw_graph(_g, vertex_text=_g.vertex_index)
# split_list = list(all_local_paths_as_splits(_g, 8))
# _g = mutate_apply_splits(_g, split_list=split_list)
# # draw_graph(_g, vertex_text=_g.vertex_index)
# split_list = list(all_local_paths_as_splits(_g, 11))
# _g = mutate_apply_splits(_g, split_list=split_list)
# # draw_graph(_g, vertex_text=_g.vertex_index)

_g = mutate_compress_all_unitigs(_g)
# # draw_graph(_g, vertex_text=_g.vertex_index)

# split_list = list(all_local_paths_as_splits(_g, 0))
# _g = mutate_apply_splits(_g, split_list=split_list)
# # draw_graph(_g, vertex_text=_g.vertex_index)

draw_graph(_g, vertex_text=_g.vertex_index)

print(get_depth_matrix(_g))
print((get_depth_matrix(_g) * _g.vp.length.a).sum())

In [ ]:
def splits_for_all_vertices(g, split_func):
    for v in g.iter_vertices():
        yield from split_func(g, v)

In [ ]:
paths = [
    [0, 1, 2, 0],
    [2, 2]
]
nvertices = 3
_g = new_graph_from_merged_paths(
    paths,
    depths=np.array([[1] * nvertices]),
    lengths=np.array([1] * nvertices),
)

draw_graph(_g, vertex_text=_g.vertex_index)
split_list = list(splits_for_all_vertices(_g, all_local_paths_as_splits))
_g = mutate_apply_splits(_g, split_list=split_list)
draw_graph(_g, vertex_text=_g.vertex_index)

In [ ]:
def mutate_split_all_nodes(g, split_func):
    # TODO: Vertices with <= 1 local path will be split into just
    # themselves pointing trivially at their neighbors.
    # These can be dropped as they are effectively a no-op.
    split_list = list(splits_for_all_vertices(g, split_func))
    g = mutate_apply_splits(g, split_list=split_list)
    return g

In [ ]:
paths = [
    [0, 1, 2, 0],
    [0, 0],
    [1, 1],
    [2, 2],
]
nvertices = 3
_g = new_graph_from_merged_paths(
    paths,
    depths=np.array([[1] * nvertices]),
    lengths=np.array([1] * nvertices),
)
draw_graph(_g, vertex_text=_g.vertex_index)
mutate_split_all_nodes(_g, split_func=all_local_paths_as_splits)
draw_graph(_g, vertex_text=_g.vertex_index)
mutate_split_all_nodes(_g, split_func=all_local_paths_as_splits)
draw_graph(_g, vertex_text=_g.vertex_index)
mutate_split_all_nodes(_g, split_func=all_local_paths_as_splits)
mutate_split_all_nodes(_g, split_func=all_local_paths_as_splits)
mutate_split_all_nodes(_g, split_func=all_local_paths_as_splits)
mutate_split_all_nodes(_g, split_func=all_local_paths_as_splits)
mutate_split_all_nodes(_g, split_func=all_local_paths_as_splits)
%prun mutate_split_all_nodes(_g, split_func=all_local_paths_as_splits)

In [ ]:
%prun mutate_compress_all_unitigs(_g)

In [ ]:
nvertices = 1_000
vs = list(range(nvertices))
paths = [
    vs,
    list(np.random.choice(vs, 200)),
]

_g = new_graph_from_merged_paths(
    paths,
    depths=np.array([[1] * nvertices]),
    lengths=np.array([1] * nvertices),
)

draw_graph(_g, vertex_text=_g.vertex_index)
%prun mutate_compress_all_unitigs(_g)
draw_graph(_g, vertex_text=_g.vertex_index)

In [ ]:
nvertices = 10_000
vs = list(range(nvertices))
paths = (
    [
        vs,  # A long genome
        list(np.random.choice(vs, 500)), # Long-range interconnects
        list(np.random.choice(vs, 500)),
        list(np.random.choice(vs, 500)),
    ]
    + [[c, c] for c in np.random.choice(vs, 500)] # Self-loops
)
_g = new_graph_from_merged_paths(
    paths,
    depths=np.array([[1] * nvertices]),
    lengths=np.array([1] * nvertices),
)
print(_g)
%prun mutate_compress_all_unitigs(_g)
print(_g)

In [ ]:
%prun mutate_split_all_nodes(_g, split_func=all_local_paths_as_splits)

In [ ]:
import matplotlib.pyplot as plt

plt.hist(_g.vp.length.a)
plt.yscale('log')

In [ ]:
plt.hist(get_depth_matrix(_g).squeeze())

In [ ]:
nvertices = 10_000
vs = list(range(nvertices))
paths = (
    [
        vs,  # A long genome
        list(np.random.choice(vs, 500)), # Long-range interconnects
        list(np.random.choice(vs, 500)),
        list(np.random.choice(vs, 500)),
    ]
    + [[c, c] for c in np.random.choice(vs, 500)] # Self-loops
)

for nsamples in [100]:
    print('\n', nsamples)
    _g = new_graph_from_merged_paths(
        paths,
        depths=np.array([np.arange(nsamples)]*nvertices).T,
        lengths=np.array([1] * nvertices),
    )
    print(_g)
    %time mutate_compress_all_unitigs(_g)
    print(_g)
    %prun mutate_split_all_nodes(_g, split_func=all_local_paths_as_splits)
    print(_g)

In [ ]:
draw_graph(g0, pos=g0_pos, vertex_text=g0.vertex_index)

In [ ]:
def total_in(f):
    return np.sum(f, axis=0)

def flow_error(f, d):
    return d - total_in(f)

def frac_contribution(f):
    return f / total_in(f)

def frac_scaled_error(f, d):
    return np.nan_to_num(frac_contribution(f) * flow_error(f, d))

def bidir_sse(f, d, weight=1):
    sq_flow_errorR = flow_error(f, d) ** 2
    sq_flow_errorL = flow_error(f.T, d) ** 2
    return (sq_flow_errorR * weight + sq_flow_errorL * weight).sum()

def bidir_mean_scaled_error(f, d):
    return (frac_scaled_error(f, d) + frac_scaled_error(f.T, d).T) / 2

def step(f, d):
    return f + bidir_mean_scaled_error(f, d)

def estimate_flow(r0, d, epsilon=1e-10, return_trace=False):
    r = r0
    loss = bidir_sse(r, d)
    r_history = [r]
    loss_history = [loss]
    while True:
        r = step(r, d)
        loss = bidir_sse(r, d)
        delta = loss_history[-1] - loss
        loss_history.append(loss)
        r_history.append(r)
        if delta < epsilon:
            if return_trace:
                return r_history, loss_history
            else:
                return r_history[-1]

In [ ]:
inverse_total_inflow = sp.sparse.csr_array(sp.sparse.diags(1 / f[1].sum(axis=0)))
frac_contribution = sp.sparse.csr_array(gt.spectral.adjacency(_g)) * inverse_total_inflow

In [ ]:
def inverse_total_inflow

In [ ]:
%%time
i = 2

d = _g.vp.depth.get_2d_array([i])
a = sp.sparse.csr_array(gt.spectral.adjacency(_g))
f_history, loss_history = estimate_flow(a, d, epsilon=1e-10, return_trace=True)

In [ ]:
def estimate_flow_all_samples(g, samples=None):
    if samples is None:
        samples = range(g.gp.nsamples)

    d = get_depth_matrix(g)
    a = sp.sparse.csr_array(gt.spectral.adjacency(g))
    flows = []
    for i in samples:
        f = sp.sparse.csr_array(estimate_flow(a, d[i], epsilon=1e-2))
        flows.append(f)
    return flows

In [ ]:
f = estimate_flow_all_samples(_g, samples=[1, 2])

In [ ]:
def get_all_edge_values_from_matrix(g, x):
    ii, jj = g.get_edges().T
    return x[jj, ii]

In [ ]:
_g.new_edge_property()

In [ ]:
_g.new_edge_property('float').a.shape

In [ ]:
_g.get_edges([p]).shape

In [ ]:
def edge_property_from_matrix(g, x):
    p = g.new_edge_property('float')
    ii, jj = g.get_edges().T
    # FIXME: Because edge properties are indexed by some non-existent, edges, it's not clear what to do here...
    p.a[:len(ii)] = get_all_edge_values_from_matrix(g, x)
    return p

In [ ]:
def get_matrix_from_edge_property(g, p):
    return sp.sparse.csr_array(gt.spectral.adjacency(g, weight=p))

x0 = f[1]
p = edge_property_from_matrix(_g, x0)
x1 = get_matrix_from_edge_property(_g, p)

x0.sum(), x1.sum()

In [ ]:
get_all_edge_values_from_matrix(_g, x).shape

In [ ]:
_g.new_edge_property('int', val=1).a.shape

In [ ]:
_g.get_edges()

In [ ]:
plt.plot(get_all_edge_values_from_matrix(_g, x0))

In [ ]:
plt.plot(get_all_edge_values_from_matrix(_g, x1))

In [ ]:
f[0][a, b].shape

In [ ]:
z = f[0].toarray()
ii, jj = _g.get_edges().T
z[jj, ii].sum()

In [ ]:
z[jj, ii]

In [ ]:
f[0].toarray().sum()

In [ ]:
_g.get_edges()[252]

In [ ]:
_g.get_in_edges(146)

In [ ]:
_g.get_out_edges(146)

In [ ]:
f[1][146, 146]

In [ ]:
x

In [ ]:
((x0.toarray() > 0) == (x1.toarray() > 0)).mean()

In [ ]:
x1#.toarray().sum()

In [ ]:
x0#.toarray().sum()

In [ ]:
get_all_edge_values_from_matrix(_g, x0)

In [ ]:
x0

In [ ]:
get_all_edge_values_from_matrix(_g, x0).shape

In [ ]:
f[1].toarray()

In [ ]:
(x0 > 0).sum()

In [ ]:
(x1 > 0).sum()

In [ ]:
(x0 > 0.5).sum()

In [ ]:
(x1 > 0.5).sum()

In [ ]:
x1.toarray()

In [ ]:
(x1 == 0.75).sum()

In [ ]:
(x0 == 0.75).toarray().sum()

In [ ]:
x1.toarray().sum(1)

In [ ]:
get_matrix_from_edge_property(_g, p).toarray().sum(1)

In [ ]:
x.toarray()

In [ ]:
x.toarray().sum()

In [ ]:
%time f[1].toarray()

In [ ]:
e = _g.new_edge_property('vector<float>')

In [ ]:
f[1].toarray().shape

In [ ]:
_g.get_edges()